<a href="https://colab.research.google.com/github/Janier1992/Python-Prep/blob/main/Aprendizaje_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV en Google Drive
ruta_archivo = "/content/drive/MyDrive/IA_IBERO/RutasBogota_T1.csv"

# Leer el archivo CSV en un DataFrame de Pandas
data = pd.read_csv(ruta_archivo, sep=";")

# Mostrar las primeras filas
print("Primeras filas del dataset:")
print(data.head())

# Verificar valores faltantes
print("\nValores faltantes por columna:")
print(data.isnull().sum())

# Separar la columna de la hora de salida en horas, minutos y segundos
data[['Hora', 'Minutos', 'Segundos']] = data['Hr Salida'].str.split(':', expand=True).astype(int)

# Contar la cantidad de veces que cada usuario viaja en la misma ruta en un día
data['Fecha'] = pd.to_datetime(data['Fecha'], dayfirst=True)
user_routes = data.groupby(['Fecha', 'usuarios', 'Ruta Salida', 'Parada']).size().reset_index(name='Count')

# Crear una característica que cuente la cantidad de viajes de un usuario en la misma ruta y parada en un día
data = data.merge(user_routes, on=['Fecha', 'usuarios', 'Ruta Salida', 'Parada'])
data['Fraude'] = data['Count'].apply(lambda x: 1 if x > 1 else 0)

# Seleccionar características y variable objetivo
features = ['Hora', 'Minutos', 'Segundos', 'Count']
X = data[features]
y = data['Fraude']

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Crear un DataFrame con las horas de salida para los primeros diez usuarios cada 2 horas
horas_salida = pd.DataFrame({
    'Hora': [10, 12, 14, 16, 18, 20, 22, 0, 2, 4],
    'Minutos': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Segundos': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Count': [1]*10  # Suponemos 1 viaje para la predicción
})

# Escalar las horas de salida
horas_salida_scaled = scaler.transform(horas_salida)

# Hacer la predicción de fraude para los primeros diez usuarios cada 2 horas
for index, row in horas_salida.iterrows():
    hora_salida_np = horas_salida_scaled[index].reshape(1, -1)
    fraude_predicho = model.predict(hora_salida_np)
    fraude_prob = model.predict_proba(hora_salida_np)[:, 1]  # Probabilidad de fraude

    prediccion_fraude = 'Sí' if fraude_predicho[0] == 1 else 'No'

    print(f"Hora de salida: {row['Hora']:02d}:{row['Minutos']:02d}:{row['Segundos']:02d}, "
          f"Predicción de fraude: {prediccion_fraude}, "
          f"Probabilidad de fraude: {fraude_prob[0]:.2f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeras filas del dataset:
       Fecha usuarios   Ruta Salida Hr Salida  Parada Tipo_Servicio
0  5/06/2023     114A  Portal Norte  10:01:30  Héroes        URBANO
1  5/06/2023     115A  Portal Norte  10:20:30  Héroes        URBANO
2  5/06/2023     116a  Portal Norte  10:37:30  Héroes        URBANO
3  5/06/2023     117a  Portal Norte  10:53:30  Héroes        URBANO
4  5/06/2023     118a  Portal Norte  11:09:30  Héroes        URBANO

Valores faltantes por columna:
Fecha            0
usuarios         0
Ruta Salida      0
Hr Salida        0
Parada           0
Tipo_Servicio    0
dtype: int64

Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00         7

    accuracy                           1.00        32
   macro avg       